In [1]:
import mlflow
import mlflow.keras
import os
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

2025-06-16 17:12:07.415936: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-16 17:12:07.420900: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-16 17:12:07.433716: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750083127.453624    6558 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750083127.459450    6558 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750083127.475490    6558 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
# Load MLflow credentials from the environment variables
MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI", "https://dagshub.com/MoraaOntita/GasSensor-DriftCompensation.mlflow")
MLFLOW_TRACKING_USERNAME = os.getenv("MLFLOW_TRACKING_USERNAME", "MoraaOntita")
MLFLOW_TRACKING_PASSWORD = os.getenv("MLFLOW_TRACKING_PASSWORD", "Musy1844")

# Configure MLflow to use the DagsHub endpoint
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

# Set up authentication (DagsHub uses basic auth for MLflow)
os.environ["MLFLOW_TRACKING_USERNAME"] = MLFLOW_TRACKING_USERNAME
os.environ["MLFLOW_TRACKING_PASSWORD"] = MLFLOW_TRACKING_PASSWORD

# Start a new MLflow run
mlflow.start_run(run_name="GasSensor_Model_Evaluation")


<ActiveRun: >

In [3]:
def evaluate_model(model, X_test, y_test):
    """
    Evaluate the model on the test data and log metrics to MLflow.
    """
    try:
        # Make predictions
        y_pred = model.predict(X_test)
        y_pred_classes = y_pred.argmax(axis=1)
        y_true_classes = y_test.argmax(axis=1)

        # Calculate accuracy
        accuracy = accuracy_score(y_true_classes, y_pred_classes)
        conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
        report = classification_report(y_true_classes, y_pred_classes)

        # Log metrics to MLflow
        mlflow.log_metric("accuracy", accuracy)

        # Log confusion matrix (you can log the image if you prefer)
        mlflow.log_text(str(conf_matrix), "confusion_matrix.txt")
        mlflow.log_text(report, "classification_report.txt")

        # Log the model to MLflow
        mlflow.keras.log_model(model, "model")

        # End the MLflow run
        mlflow.end_run()

        print(f"Model evaluation completed. Accuracy: {accuracy}")

    except Exception as e:
        mlflow.end_run(status='FAILED')
        print(f"Error in model evaluation: {e}")
        raise
